In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from sklearn.feature_extraction.text import CountVectorizer
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms
from sklearn.linear_model import LogisticRegression
from gensim.models import Word2Vec

def CleanText(text):
    text = text.lower() #Turn all text entries into lower-case
    text = re.sub(r'''(https?:\/\/www\.|https?:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,3}[-a-zA-Z0-9()@:%_\+.~#?&\//=<>]*''', "<URL>", text)
    #Replace URL with tag
    text = re.sub(r'''[0-9]+[/\-.]+[0-9]+[/\-.]+[0-9]+''', "<DATE>", text) #Replace dates with tag
    text = re.sub(r'''[a-z0-9._%+-]+\@[a-z0-9.-]+[a-z0-9]\.[a-z]{1,}''', "<EMAIL>", text)
    text = re.sub(r'''[0-9]+''', "<NUM>", text) #Replace numbers with tag
    
    text = re.sub(r'''[.|,|!|?|\'|\''|\"|\n|\t|\-|\(|\)]''', '', text)
    text = re.sub(r'''^\s+|\s+$''', '', text) #Remove whitespaces at the end of string
    text = re.sub(r'''[ ][ ]+|_''', " ", text) #Remove multiple whitespace
    return text

df = pd.read_csv("all_data_with_identities.csv")

# Find columns needed

In [9]:
df = df.loc[:, ["comment_text", "split", "na_gender", "toxicity", "male", "female", "transgender", "other_gender"]]


In [10]:
training_data = df[df['split'] == 'train']
training_data = training_data[training_data['na_gender'] == 0][:5000]

test_data = df[df['split'] == 'test'][:500]
validation_data = df[df['split'] == 'val']

training_data['comment_text'] = training_data['comment_text'].apply(lambda text: CleanText(text))
training_data['male'] = training_data['male'].apply(lambda x: round(x))
training_data['female'] = training_data['female'].apply(lambda x: round(x))
training_data['transgender'] = training_data['transgender'].apply(lambda x: round(x))
training_data['other_gender'] = training_data['transgender'].apply(lambda x: round(x))

test_data['comment_text'] = test_data['comment_text'].apply(lambda text: CleanText(text))


In [11]:
trainingComments = training_data['comment_text']
trainingLabels = training_data['toxicity']
trainingComments = trainingComments.values.tolist()

testComments = test_data['comment_text']
testComments = testComments.values.tolist()
testLabels = test_data['toxicity']

valComments = validation_data['comment_text']
valComments = valComments.values.tolist()
valLabels = validation_data['toxicity']



In [12]:
# Using count vectorizer with predefined stopwords
cv = CountVectorizer(stop_words='english')

# Creating vocabulary by fitting on trainingdata and transforming to the vocabulary vector
count_vectorTrainingSet = cv.fit_transform(trainingComments)

# Transforming test data to the vocabulary vector
count_vectorTestSet = cv.transform(testComments)

# Using the count_vector to create a matrix containing vectors of each review
def createVectorArr(count_vector):    
    vectors = []
    for i in range(len(count_vector.toarray())):
        vectors.append(count_vector[i].toarray()[0])
    vectors = np.array(vectors)
    return vectors

X_train_vectors = createVectorArr(count_vectorTrainingSet)
X_test_vectors = createVectorArr(count_vectorTestSet)

for i, ind in enumerate(training_data.index):
    training_data['comment_text'][ind] = X_train_vectors[i]
    
for i, ind in enumerate(test_data.index):
    test_data['comment_text'][ind] = X_test_vectors[i]

<ipython-input-12-870a0a205244>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['comment_text'][ind] = X_train_vectors[i]
<ipython-input-12-870a0a205244>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['comment_text'][ind] = X_test_vectors[i]


In [13]:
training_data

,comment_text,split,na_gender,toxicity,male,female,transgender,other_gender
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train,0,0.545455,1,1,0,0
13,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train,0,0.800000,1,0,0,0
21,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train,0,0.363636,1,0,0,0
31,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train,0,0.800000,1,1,0,0
46,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",train,0,0.594595,0,1,0,0
...,...,...,...,...,...,...,...,...
43090,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train,0,0.000000,1,0,0,0
43097,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train,0,0.000000,1,1,0,0
43107,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train,0,0.000000,0,1,0,0
43114,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",train,0,0.000000,1,1,0,0


In [24]:
class AdalineGD():
    def __init__(self, learning_rate = 0.01, n_iter = 20, w = None):
        self.w = w
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        
    def CheckAccuracy(self, predictions, labels):
        acc = 0.0
        for i in range(len(predictions)):
            if (predictions[i] == round(labels[i])):
                acc += 1
        return acc/len(predictions)
    
    def predict(self, X_test):
        # Making sure that array is numpy array
        X_test = np.array(X_test).reshape((len(X_test), -1))
        # Using the sign function as activation function since labels are {1, -1}
        preds = np.dot(X_test, self.w)
        
        for i in range(len(preds)):
            if preds[i] <= 0:
                preds[i] = 0
            else:
                preds[i] = 1
        return preds
        
    def fit(self, df):
        # Making sure that arrays are numpy arrays
        
        X_train = df['comment_text'].values.tolist()
        Y_train = df['toxicity'].values.tolist()
        
        X_train = np.array(X_train).reshape((len(X_train), -1))
        Y_train = np.array(Y_train).reshape((len(Y_train), 1))
        
        
        # Initializing w vector using random normal distribution
        if self.w == None:
            self.w = np.random.normal(0, 0.1, len(X_train[0])).reshape(len(X_train[0]), 1)
            #self.w = np.zeros(len(X_train[0])).reshape(len(X_train[0]), 1)
        
        print("nice")
        
        # Updating w vector for each sample
        output = np.dot(X_train, self.w)
        error = (Y_train - output)            
        self.w += (1/len(Y_train)) * self.learning_rate * np.dot(X_train.T, error)
        
        dic = {}
        for col in df.columns[4:]:
            tempSet = df[df[col] == 1]
            tempLabels = tempSet['toxicity'].values.tolist()
            tempVecs = tempSet['comment_text'].values.tolist()
            dic[col] = [tempVecs, tempLabels]
        
        # Using n epochs
        for i in range(self.n_iter):
            accuracies = []
            for col in df.columns[4:]:
                tempPredictions = self.predict(dic[col][0])
                currentAcc = self.CheckAccuracy(tempPredictions, dic[col][1])
                accuracies.append([currentAcc, col])
            
            accuracies = np.array(accuracies)
            if i == 0:
                print(accuracies)
            
            worstCol = accuracies[np.argmin(accuracies[:,0])][1]
            print(accuracies[np.argmin(accuracies[:,0])])
            X_train = dic[worstCol][0]
            Y_train = dic[worstCol][1]
            
            X_train = np.array(X_train).reshape((len(X_train), -1))
            Y_train = np.array(Y_train).reshape((len(Y_train), 1))
            
            # Updating w vector for each sample
            output = np.dot(X_train, self.w)
            error = (Y_train - output)
            self.w += (1/len(Y_train)) * self.learning_rate * np.dot(X_train.T, error)
        print(accuracies)
        return self
    
    

In [25]:
model = AdalineGD(learning_rate = 0.5, n_iter = 20)

model.fit(training_data)

predictions = model.predict(X_test_vectors)

diller = np.array(testLabels.values.tolist())

model.CheckAccuracy(predictions, diller)




nice
[['0.5497553017944535' 'male']
 ['0.5133884297520661' 'female']
 ['0.5231788079470199' 'transgender']
 ['0.5231788079470199' 'other_gender']]
['0.5133884297520661' 'female']
['0.5099337748344371' 'transgender']
['0.5345454545454545' 'female']
['0.5325619834710744' 'female']
['0.5464462809917355' 'female']
['0.5467768595041322' 'female']
['0.5514049586776859' 'female']
['0.5533884297520661' 'female']
['0.555702479338843' 'female']
['0.5586776859504132' 'female']
['0.5596694214876033' 'female']
['0.5629752066115703' 'female']
['0.5639669421487603' 'female']
['0.5616528925619835' 'female']
['0.5659504132231405' 'female']
['0.5685950413223141' 'female']
['0.5705785123966942' 'female']
['0.5732231404958678' 'female']
['0.5752066115702479' 'female']
['0.5765289256198347' 'female']
[['0.5815660685154975' 'male']
 ['0.5765289256198347' 'female']
 ['0.6026490066225165' 'transgender']
 ['0.6026490066225165' 'other_gender']]


0.638

In [ ]:
# Preparing reviews in list of lists format
X_trainForw2v = []
X_testForw2v = []

for sentence in X_train:
    X_trainForw2v.append(sentence.split(' '))

for sentence in X_test:
    X_testForw2v.append(sentence.split(' '))

X_trainForw2v = np.array(X_trainForw2v)
X_testForw2v = np.array(X_testForw2v)

In [11]:
# Do not want words with overall less than 3 appearances to count, generating vector of size 200 for each word
modelw2v = Word2Vec(X_trainForw2v, min_count=3, size=100, iter=60)

In [12]:
def TransformToEmbedding(model, data):
    transformed = np.zeros((len(data), model.vector_size))
    for i, sentence in enumerate(data):
        currentLst = []
        for word in sentence:
            if word in model.wv.vocab.keys():
                currentLst.append(model.wv[word])
        if (len(currentLst) > 0):
            currentLst = np.mean(np.array(currentLst), axis=0)
        else:
            currentLst = np.zeros(model.vector_size)
        transformed[i] = np.array(currentLst)
    return np.array(transformed)


X_trainForw2vTransformed = TransformToEmbedding(modelw2v, X_trainForw2v)
X_testForw2vTransformed = TransformToEmbedding(modelw2v, X_testForw2v)

In [14]:

# Using sklearn linear Logistic regression
clf1 = LogisticRegression(random_state=0)
clf1.fit(X_trainForw2vTransformed, Y_train)
predictions = clf1.predict(X_testForw2vTransformed)
print("Accuracy on TestSet: ", CheckAccuracy(predictions, Y_test))



Accuracy on TestSet:  0.7345
